In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import numpy as np
import json
from PIL import Image
import time
import math
from datetime import datetime
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

TRAIN = True #True for training, False to load the model
SEED = 1234
tf.random.set_seed(SEED)  
cwd = os.getcwd()
num_classes = 3
bs = 1
img_h = 512
img_w = 512
up_h = 1024
up_w = 1024 
orig_img_h = 1536
orig_img_w = 2048

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def IoU(y_true, y_pred):
  score_th = 0.5
  y_pred = tf.cast(y_pred > score_th, tf.float32)
    
  intersection = tf.reduce_sum(y_true * y_pred)
  union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    
  return ((intersection + 1e-7) / (union + 1e-7))

In [ ]:
# Load the Preliminary models
modelWeed = tf.keras.models.load_model('/content/drive/My Drive/ANN/Models/Preliminary_UNET_weed', custom_objects={'IoU':IoU})
modelCrop = tf.keras.models.load_model('/content/drive/My Drive/ANN/Models/Preliminary_UNET_crop', custom_objects={'IoU':IoU})

In [ ]:
# CustomDataset
# -------------

# Custom generator the yelds the RGB image and the converted mask (only with weed or crop)
# in order to have the masks of the training/validation dataset predicted by the two 
# Preliminary networks
class CustomDataset(tf.keras.utils.Sequence):
  def __init__(self, dataset_dir, which_subset, img_generator=None, mask_generator=None, 
               preprocessing_function=None, out_shape=[img_h, img_w], cult_type=None):
    if which_subset == 'training':
      subset_file = os.path.join(dataset_dir, 'Splits', 'train.txt')
    elif which_subset == 'validation':
      subset_file = os.path.join(dataset_dir, 'Splits', 'val.txt')
    
    with open(subset_file, 'r') as f:
      lines = f.readlines()
    
    subset_filenames = []
    for line in lines:
      subset_filenames.append(line.strip()) 

    self.which_subset = which_subset
    self.dataset_dir = dataset_dir
    self.subset_filenames = subset_filenames
    self.img_generator = img_generator
    self.mask_generator = mask_generator
    self.preprocessing_function = preprocessing_function
    self.out_shape = out_shape
    self.cult_type = cult_type

  def __len__(self):
    return len(self.subset_filenames)

  def _read_rgb_mask(self, img_path):
    mask_img = Image.open(img_path)
    mask_img = mask_img.resize(self.out_shape, resample=Image.NEAREST)
    mask_arr = np.array(mask_img)

    new_mask_arr = np.zeros(mask_arr.shape[:2], dtype=mask_arr.dtype)

    # Use RGB dictionary in 'RGBtoTarget.txt' to convert RGB to target
    if self.cult_type == 'weed':
      new_mask_arr[np.where(np.all(mask_arr == [216, 67, 82], axis=-1))] = 1
    
    elif self.cult_type == 'crop':
      new_mask_arr[np.where(np.all(mask_arr == [255, 255, 255], axis=-1))] = 1

    return new_mask_arr

  def __getitem__(self, index):
    curr_filename = self.subset_filenames[index]
    img = Image.open(os.path.join(self.dataset_dir, 'Images', curr_filename + '.jpg'))
    img = img.resize(self.out_shape)

    img_arr = np.array(img)

    mask_arr = self._read_rgb_mask(os.path.join(self.dataset_dir, 'Masks', curr_filename + '.png'))

    mask_arr = np.expand_dims(mask_arr, -1)
    
    if self.preprocessing_function is not None:
        img_arr = self.preprocessing_function(img_arr)

    return img_arr, np.float32(mask_arr)

In [ ]:
# Creation of the training and validation generators for the Preliminary Networks
dataset_dir = os.path.join(cwd, 'drive/MyDrive/Assignment_2/Dataset')
dataset_dir = os.path.join(dataset_dir, 'BipBip')

dataset_crop = CustomDataset(dataset_dir, 'training', cult_type='crop',img_generator=None, mask_generator=None)

dataset_valid_crop = CustomDataset(dataset_dir, 'validation', cult_type='crop', img_generator=None, mask_generator=None)

train_dataset_crop = tf.data.Dataset.from_generator(lambda: dataset_crop,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([img_h, img_w, 3], [img_h, img_w, 1]))
train_dataset_crop = train_dataset_crop.batch(bs)

valid_dataset_crop = tf.data.Dataset.from_generator(lambda: dataset_valid_crop,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([img_h, img_w, 3], [img_h, img_w, 1]))
valid_dataset_crop = valid_dataset_crop.batch(bs)

dataset_weed = CustomDataset(dataset_dir, 'training', cult_type='weed',img_generator=None, mask_generator=None)

dataset_valid_weed = CustomDataset(dataset_dir, 'validation', cult_type='weed',img_generator=None, mask_generator=None)

train_dataset_weed = tf.data.Dataset.from_generator(lambda: dataset_weed,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([img_h, img_w, 3], [img_h, img_w, 1]))
train_dataset_weed = train_dataset_weed.batch(bs)

valid_dataset_weed = tf.data.Dataset.from_generator(lambda: dataset_valid_weed,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([img_h, img_w, 3], [img_h, img_w, 1]))
valid_dataset_weed = valid_dataset_weed.batch(bs)

In [ ]:
# Creation of the masks to use in the training phase, respectively:
# Train_crop, Train_weed, Valid_crop, Valid_weed
train_crop_masks = modelCrop.predict(train_dataset_crop)
result_train_crop = np.array(train_crop_masks)
print('crop_train', result_train_crop.shape)
valid_crop_masks = modelCrop.predict(valid_dataset_crop)
result_valid_crop = np.array(valid_crop_masks)
print('crop_valid', result_valid_crop.shape)
train_weed_masks = modelWeed.predict(train_dataset_weed)
result_train_weed = np.array(train_weed_masks)
print('weed_train', result_train_weed.shape)
valid_weed_masks = modelWeed.predict(valid_dataset_weed)
result_valid_weed = np.array(valid_weed_masks)
print('weed_valid', result_valid_weed.shape)

crop_train (72, 512, 512, 1)
crop_valid (18, 512, 512, 1)
weed_train (72, 512, 512, 1)
weed_valid (18, 512, 512, 1)


In [ ]:
# CustomDatasetMasks
# -------------

# Custom generator the yelds a five layer input (first 3 are the RGB image, the last 2 are the mask 
# generated by the preliminary networks) and the output mask
class CustomDatasetMask(tf.keras.utils.Sequence):
  def __init__(self, dataset_dir, which_subset, out_shape=[up_h, up_w], cult_type=None, weed=None, crop=None):
    if which_subset == 'training':
      subset_file = os.path.join(dataset_dir, 'Splits', 'train.txt')
    elif which_subset == 'validation':
      subset_file = os.path.join(dataset_dir, 'Splits', 'val.txt')
    
    with open(subset_file, 'r') as f:
      lines = f.readlines()
    
    subset_filenames = []
    for line in lines:
      subset_filenames.append(line.strip()) 

    self.which_subset = which_subset
    self.dataset_dir = dataset_dir
    self.subset_filenames = subset_filenames
    self.out_shape = out_shape
    self.cult_type = cult_type
    self.weed = weed
    self.crop = crop 

  def __len__(self):
    return len(self.subset_filenames)

  def _read_rgb_mask(self, img_path):
    mask_img = Image.open(img_path)
    mask_img = mask_img.resize(self.out_shape, resample=Image.NEAREST)
    mask_arr = np.array(mask_img)

    new_mask_arr = np.zeros(mask_arr.shape[:2], dtype=mask_arr.dtype)

    # Use RGB dictionary in 'RGBtoTarget.txt' to convert RGB to target
    if self.cult_type == 'weed':
      new_mask_arr[np.where(np.all(mask_arr == [216, 67, 82], axis=-1))] = 1
    
    elif self.cult_type == 'crop':
      new_mask_arr[np.where(np.all(mask_arr == [255, 255, 255], axis=-1))] = 1
    
    elif self.cult_type == 'merge':
      new_mask_arr[np.where(np.all(mask_arr == [255, 255, 255], axis=-1))] = 1
      new_mask_arr[np.where(np.all(mask_arr == [216, 67, 82], axis=-1))] = 2

    return new_mask_arr

  def __getitem__(self, index):
    curr_filename = self.subset_filenames[index]
    mask_arr = self._read_rgb_mask(os.path.join(self.dataset_dir, 'Masks', curr_filename + '.png'))
    mask_arr = np.expand_dims(mask_arr, -1) 

    img = Image.open(os.path.join(self.dataset_dir, 'Images', curr_filename + '.jpg'))
    img = img.resize(self.out_shape)
    img_arr = np.array(img)

    return (img_arr, self.weed[index], self.crop[index]), np.float32(mask_arr)

In [ ]:
# ImageDataGenerator
# ------------------
apply_data_augmentation = True

if apply_data_augmentation:
    img_data_gen = ImageDataGenerator(rotation_range=30,
                                      width_shift_range=30,
                                      height_shift_range=30,
                                      zoom_range=0.4,
                                      horizontal_flip=True,
                                      vertical_flip=True,
                                      fill_mode='reflect')
    mask_data_gen = ImageDataGenerator(rotation_range=30,
                                       width_shift_range=30,
                                       height_shift_range=30,
                                       zoom_range=0.4,
                                       horizontal_flip=True,
                                       vertical_flip=True,
                                       fill_mode='reflect')

In [ ]:
# Creation of the training and validation generators for the Merge Network
dataset_dir = os.path.join(cwd, 'drive/MyDrive/Assignment_2/Dataset')
dataset_dir = os.path.join(dataset_dir, 'BipBip')

dataset_train_img = CustomDatasetMask(dataset_dir, 'training', cult_type='merge', weed=result_train_weed, crop=result_train_crop)
dataset_valid_img = CustomDatasetMask(dataset_dir, 'validation', cult_type='merge', weed=result_valid_weed, crop=result_valid_crop)

types = ((tf.float32, tf.float32, tf.float32), tf.float32)
shapes = (([up_h, up_w, 3], [img_h, img_w, 1], [img_h, img_w, 1]), [up_h, up_w, 1])

train_img = tf.data.Dataset.from_generator(lambda: dataset_train_img, output_types=types ,output_shapes=shapes)
train_img = train_img.batch(bs)
train_img = train_img.repeat()
valid_img = tf.data.Dataset.from_generator(lambda: dataset_valid_img, output_types=types ,output_shapes=shapes)
valid_img = valid_img.batch(bs)
valid_img = valid_img.repeat()

In [ ]:
# Creation of the model of the network:
# it's a custom version of the UNet, with Conv2D (with He-uniform initialization), 
# BatchNormalization and LeakyReLu layers
def MERGE(input_size = (up_h,up_w,5)):
    image_input = keras.Input(shape=(up_h, up_w, 3), name="img_input")
    weed_mask = keras.Input(shape=(img_h, img_w, 1), name="weed_input")
    weed_upscale = keras.layers.UpSampling2D(size = (2,2))(weed_mask)
    crop_mask = keras.Input(shape=(img_h, img_w, 1), name="crop_input")
    crop_upscale = keras.layers.UpSampling2D(size = (2,2))(crop_mask)
    inputs = keras.layers.concatenate([image_input,weed_upscale,crop_upscale], axis=-1)
    
    conv1 = keras.layers.Conv2D(16, 3, padding='same', kernel_initializer = 'he_uniform')(inputs)
    bano1 = keras.layers.BatchNormalization()(conv1)
    acti1 = keras.layers.LeakyReLU()(bano1)
    pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(acti1)

    conv2 = keras.layers.Conv2D(32, 3, padding='same', kernel_initializer = 'he_uniform')(pool1)
    bano2 = keras.layers.BatchNormalization()(conv2)
    acti2 = keras.layers.LeakyReLU()(bano2)
    pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(acti2)

    conv3 = keras.layers.Conv2D(64, 3, padding='same', kernel_initializer = 'he_uniform')(pool2)
    bano3 = keras.layers.BatchNormalization()(conv3)
    acti3 = keras.layers.LeakyReLU()(bano3)
    pool3 = keras.layers.MaxPooling2D(pool_size=(2, 2))(acti3)

    conv4 = keras.layers.Conv2D(128, 3, padding='same', kernel_initializer = 'he_uniform')(pool3)
    bano4 = keras.layers.BatchNormalization()(conv4)
    acti4 = keras.layers.LeakyReLU()(bano4)
    pool4 = keras.layers.MaxPooling2D(pool_size=(2, 2))(acti4)

    conv5 = keras.layers.Conv2D(256, 3, padding='same', kernel_initializer = 'he_uniform')(pool4)
    bano5 = keras.layers.BatchNormalization()(conv5)
    acti5 = keras.layers.LeakyReLU()(bano5)
    pool5 = keras.layers.MaxPooling2D(pool_size=(2, 2))(acti5)

    ###################################################################################################################
    conv6 = keras.layers.Conv2D(512, 3, padding = 'same', kernel_initializer = 'he_uniform')(pool5)
    bano6 = keras.layers.BatchNormalization()(conv6)
    acti6 = keras.layers.LeakyReLU()(bano6)
    ###################################################################################################################

    u0=keras.layers.UpSampling2D(size = (2,2))(acti6)
    c0 = keras.layers.Conv2D(256, 2, padding='same', kernel_initializer = 'he_uniform')(u0)
    c0 = keras.layers.concatenate([acti5,c0], axis = 3)
    c0 = keras.layers.Conv2D(256, 3, padding='same', kernel_initializer = 'he_uniform')(c0)
    b0 = keras.layers.BatchNormalization()(c0)
    a0 = keras.layers.LeakyReLU()(b0)

    u1=keras.layers.UpSampling2D(size = (2,2))(a0)
    c1 = keras.layers.Conv2D(128, 2, padding='same', kernel_initializer = 'he_uniform')(u1)
    c1 = keras.layers.concatenate([acti4,c1], axis = 3)
    c1 = keras.layers.Conv2D(128, 3, padding='same', kernel_initializer = 'he_uniform')(c1)
    b1 = keras.layers.BatchNormalization()(c1)
    a1 = keras.layers.LeakyReLU()(b1)

    u2=keras.layers.UpSampling2D(size = (2,2))(a1)
    c2 = keras.layers.Conv2D(64, 2, padding='same', kernel_initializer = 'he_uniform')(u2)
    c2 =keras.layers.concatenate([acti3,c2], axis = 3)
    c2 = keras.layers.Conv2D(64, 3, padding='same', kernel_initializer = 'he_uniform')(c2)
    b2 = keras.layers.BatchNormalization()(c2)
    a2 = keras.layers.LeakyReLU()(b2)

    u3=keras.layers.UpSampling2D(size = (2,2))(a2)
    c3 = keras.layers.Conv2D(32, 2, padding='same', kernel_initializer = 'he_uniform')(u3)
    c3 = keras.layers.concatenate([acti2,c3], axis = 3)
    c3 = keras.layers.Conv2D(32, 3, padding='same', kernel_initializer = 'he_uniform')(c3)
    b3 = keras.layers.BatchNormalization()(c3)
    a3 = keras.layers.LeakyReLU()(b3)

    u4=keras.layers.UpSampling2D(size = (2,2))(a3)
    c4 = keras.layers.Conv2D(16, 2, padding='same', kernel_initializer = 'he_uniform')(u4)
    c4 =keras.layers.concatenate([acti1,c4], axis = 3)
    c4 = keras.layers.Conv2D(16, 3, padding='same', kernel_initializer = 'he_uniform')(c4)
    b4 = keras.layers.BatchNormalization()(c4)
    a4 = keras.layers.LeakyReLU()(b4)
    out =keras.layers.Conv2D(num_classes, 1, activation = 'softmax', name="output")(a4)

    model = keras.models.Model(inputs=[image_input,weed_mask,crop_mask], outputs=out)
    return model

In [ ]:
model = MERGE()
tf.keras.utils.plot_model(model)

In [ ]:
# Loss
# Sparse Categorical Crossentropy to use integers (mask) instead of one-hot encoded labels
loss = tf.keras.losses.SparseCategoricalCrossentropy()
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Here we define the intersection over union for each class in the batch.
# Then we compute the final iou as the mean over classes
def meanIoU(y_true, y_pred):
    # get predicted class from softmax
    y_pred = tf.expand_dims(tf.argmax(y_pred, -1), -1)
    weights = [1, 1]
    weights_sum = sum(weights)
    per_class_iou = []

    for i in range(1,num_classes): # exclude the background class 0
      # Get prediction and target related to only a single class (i)
      class_pred = tf.cast(tf.where(y_pred == i, 1, 0), tf.float32)
      class_true = tf.cast(tf.where(y_true == i, 1, 0), tf.float32)
      intersection = tf.reduce_sum(class_true * class_pred)
      union = tf.reduce_sum(class_true) + tf.reduce_sum(class_pred) - intersection
    
      iou = (weights[i-1] * (intersection + 1e-7)) / ((union + 1e-7) * weights_sum)
      per_class_iou.append(iou)

    return tf.reduce_sum(per_class_iou)
# Validation metrics
# ------------------
metrics = ['accuracy', meanIoU]
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir /content/drive/My\ Drive/ANN/Log

In [ ]:
if(TRAIN):
  exps_dir = os.path.join(cwd, 'drive/My Drive/ANN/Log/Waterfall')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S')

  model_name = 'MERGE_CoBaLeRu'

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                    save_weights_only=True, save_best_only=True)  # False to save the model directly
  callbacks.append(ckpt_callback)

  # Visualize Learning on Tensorboard
  # ---------------------------------
  tb_dir = os.path.join(exp_dir, 'tb_logs')
  if not os.path.exists(tb_dir):
      os.makedirs(tb_dir)
      
  # By default shows losses and metrics for both training and validation
  tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                              profile_batch=0,
                                              histogram_freq=0)  # if 1 shows weights histograms
  callbacks.append(tb_callback)

  reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1, cooldown=0)
  callbacks.append(reduce_lr)

  # Early Stopping
  # --------------
  early_stop = True
  if early_stop:
      es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
      callbacks.append(es_callback)



  model.fit(train_img,
            epochs=100,  #### set repeat in training dataset
            steps_per_epoch=len(dataset_train_img),
            validation_data=valid_img,
            validation_steps=len(dataset_valid_img),
            callbacks=callbacks)

In [ ]:
if(TRAIN):
  # Save the model
  model.save('/content/drive/My Drive/ANN/Models/MERGE')
else:
  model = tf.keras.models.load_model('/content/drive/My Drive/ANN/Models/MERGE', custom_objects={'meanIoU':meanIoU})

In [ ]:
# CustomTest
# -------------

# Custom generator the yelds a five layer input (first 3 are the RGB image, the last 2 are the mask 
# generated by the preliminary networks) for the testing phase
class CustomTest(tf.keras.utils.Sequence):
  def __init__(self, dataset_dir, filenames, out_shape=[up_h, up_w], weed=None, crop=None):
    self.dataset_dir = dataset_dir
    self.filenames = filenames
    self.out_shape = out_shape
    self.weed = weed
    self.crop = crop 

  def __len__(self):
    return len(self.filenames)

  def __getitem__(self, index):
    curr_filename = self.filenames[index] 
    mypath = os.path.join(self.dataset_dir, 'Images', curr_filename + '.jpg') 
    if os.path.exists(mypath):
      img = Image.open(os.path.join(self.dataset_dir, 'Images', curr_filename + '.jpg'))
    else :
      img = Image.open(os.path.join(self.dataset_dir, 'Images', curr_filename + '.png'))
    img = img.resize(self.out_shape)
    img_arr = np.array(img)

    return (img_arr, self.weed[index], self.crop[index])

In [ ]:
# Utility functions to produce the output file for the challenge
def rle_encode(img):
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

################################################################################
################################################################################

def update_dictionary(submission_dict, img, img_name, team, crop):
  submission_dict[img_name] = {}
  submission_dict[img_name]['shape'] = img.shape
  submission_dict[img_name]['team'] = team
  submission_dict[img_name]['crop'] = crop
  submission_dict[img_name]['segmentation'] = {}

  # RLE encoding  
  # crop
  rle_encoded_crop = rle_encode(img == 1)
  # weed
  rle_encoded_weed = rle_encode(img == 2)

  submission_dict[img_name]['segmentation']['crop'] = rle_encoded_crop
  submission_dict[img_name]['segmentation']['weed'] = rle_encoded_weed
  # Finally, save the results into the submission.json file
  return submission_dict

################################################################################
################################################################################

def trim_filenames(test_gen):
  filenames = test_gen.filenames
  prova = [e[7:] for e in filenames]
  filenames = []
  for file in prova:
    filenames.append(file.split(".")[0])
  return filenames

################################################################################
################################################################################

def create_prediction(category, dictionary):
  team = category.split('/')[0]
  crop = category.split('/')[1]
  path = os.path.join(cwd, '/content/drive/My Drive/Assignment_2/Dataset/All/Test_Dev', category)
  print(path)
  test_data_gen1 = ImageDataGenerator()
  test_gen1 = test_data_gen1.flow_from_directory(path,
                                               target_size=(img_h,img_w),
                                               color_mode="rgb",
                                               batch_size=1, 
                                               class_mode='categorical',
                                               classes=None,
                                               shuffle=False,
                                               seed=SEED)
  
  test_data_gen2 = ImageDataGenerator()
  test_gen2 = test_data_gen2.flow_from_directory(path,
                                               target_size=(img_h,img_w),
                                               color_mode="rgb",
                                               batch_size=1, 
                                               class_mode='categorical',
                                               classes=None,
                                               shuffle=False,
                                               seed=SEED)
  
  filenames = trim_filenames(test_gen1)
  weed_pred = modelWeed.predict(test_gen1)
  weed_pred = np.array(weed_pred)
  crop_pred = modelCrop.predict(test_gen2)
  crop_pred = np.array(crop_pred)
  test = CustomTest(path, filenames=filenames, weed=weed_pred, crop=crop_pred)
  test_img = tf.data.Dataset.from_generator(lambda: test, 
                                            output_types=(tf.float32, tf.float32, tf.float32),
                                            output_shapes=([up_h, up_w, 3], [img_h, img_w, 1], [img_h, img_w, 1]))
  test_img = test_img.batch(bs)
  iterator = iter(test_img)
  for i in range(len(filenames)):
    pred = model.predict(iterator.next())
    arr_ = np.squeeze(pred)
    img = cv2.resize(arr_, dsize=(orig_img_w, orig_img_h), interpolation=cv2.INTER_CUBIC)
    predicted_class = tf.argmax(img, -1)
    result = np.array(predicted_class)
    dictionary = update_dictionary(submission_dict=dictionary, img=result, img_name=filenames[i], team=team, crop=crop)
  return dictionary

################################################################################
################################################################################

def iterate_dataset(directories_path):
  with open(directories_path, 'r') as f:
    paths = [line.rstrip() for line in f.readlines()]
    dictionary = {}
    for name in paths:
      dictionary = create_prediction(name, dictionary)
    with open(os.path.join(cwd, 'drive/MyDrive/ANN/submission.json'), 'w') as f:
      json.dump(dictionary, f)

In [ ]:
# Creation of the JSON output file
iterate_dataset(os.path.join(cwd, '/content/drive/My Drive/ANN/directories.txt'))